In [1]:
import os
import joblib
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import tensorflow as tf


2024-10-11 22:04:29.309119: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-11 22:04:29.363993: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-11 22:04:30.244004: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
def c6_data():
    df = pd.read_csv('Fill.csv')
    df.drop_duplicates(inplace=True)

    x = df.iloc[:, :2].values
    y = df.iloc[:, 2].values

    # Normalize
    scaler_x = StandardScaler()
    x = scaler_x.fit_transform(x)

    scaler_y = StandardScaler()
    y = y.reshape(-1, 1)
    y = scaler_y.fit_transform(y)

    # Split dataset
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=666)

    return x_train, x_test, y_train, y_test, scaler_x, scaler_y

# Get data
x_train, x_test, y_train, y_test, scaler_x, scaler_y = c6_data()


In [3]:
# Assume each sample has 10 time steps
sequence_length = 10

# Generate time series data
def create_sequences(data, seq_length):
    sequences = []
    for i in range(len(data) - seq_length + 1):
        sequences.append(data[i:i + seq_length])
    return np.array(sequences)

# Create training and testing sequences
x_train_seq = create_sequences(x_train, sequence_length)
x_test_seq = create_sequences(x_test, sequence_length)

# Adjust y_train and y_test shapes to match sequences
y_train_seq = y_train[sequence_length - 1:]
y_test_seq = y_test[sequence_length - 1:]


In [4]:
# Load the complete model
model_path = './lstmmodel/model_lstm_weight'
if os.path.exists(model_path):
    model = tf.keras.models.load_model(model_path)
    print("Model loaded from disk.")

    # Predict and calculate additional metrics
    y_pred = model.predict(x_test_seq)

    # Calculate Mean Squared Error (MSE)
    mse = mean_squared_error(y_test_seq, y_pred)
    print(f"MSE: {mse}")

    # Calculate Root Mean Squared Error (RMSE)
    rmse = np.sqrt(mse)
    print(f"RMSE: {rmse}")

    # Calculate Mean Absolute Error (MAE)
    mae = mean_absolute_error(y_test_seq, y_pred)
    print(f"MAE: {mae}")

    # Calculate R²
    r2 = r2_score(y_test_seq, y_pred)
    print(f"R²: {r2}")
else:
    print("Model file not found.")
    

2024-10-11 22:04:36.091676: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 658 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:3e:00.0, compute capability: 7.5


Model loaded from disk.
  93/1805 [>.............................] - ETA: 2s

2024-10-11 22:04:38.799142: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600


1805/1805 [==============================] - 3s 2ms/step
MSE: 0.9659130492105209
RMSE: 0.9828087551556106
MAE: 0.8247754513144215
R²: 0.0354995633609928
